In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Randome Forest

In [2]:
df = pd.read_csv("final_data_Apr10.csv",index_col=0)

In [30]:
df1 = pd.read_csv("data_final.csv")

In [32]:
df1.Time

0      2020-10-02 16:30:00
1      2020-10-02 17:30:00
2      2020-10-02 18:30:00
3      2020-10-02 19:30:00
4      2020-10-02 20:30:00
              ...         
799    2021-04-01 18:30:00
800    2021-04-02 02:30:00
801    2021-04-02 05:30:00
802    2021-04-02 10:30:00
803    2021-04-02 14:30:00
Name: Time, Length: 804, dtype: object

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 0 to 797
Data columns (total 35 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   viewCount                                         798 non-null    float64
 1   likeCount                                         798 non-null    float64
 2   dislikeCount                                      798 non-null    float64
 3   num_days_since_pub                                798 non-null    float64
 4   viewcounts_weighted                               798 non-null    float64
 5   like_count_weighted                               798 non-null    float64
 6   dislike_count_weighted                            798 non-null    float64
 7   like_dislike_ratio                                798 non-null    float64
 8   weighted_like_dislike_ratio                       798 non-null    float64
 9   like_dislike_ratio_by

In [3]:
X = df.iloc[:,:34]
y = df.iloc[:,34]

In [ ]:
plt.figure(figsize=(30,15))
sns.heatmap(df.corr(), annot = True, vmin=-1, vmax=1, center= 0)

In [4]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scale = scaler.transform(X)

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
f1 = []
auc = []
kf = StratifiedKFold(n_splits=10)
X_1 = X_scale
y_1= np.array(y)
for train_index, test_index in kf.split(X_1,y_1):
    X_train, X_test = X_1[train_index], X_1[test_index]
    y_train, y_test = y_1[train_index], y_1[test_index]
    clf = RandomForestClassifier(n_estimators=400).fit(X_train, y_train)
    ypred = clf.predict(X_test)
    #print(multilabel_confusion_matrix(y_test, ypred,
    #                        labels=["buy", "hold", "sell"]))
    f1.append(f1_score(y_test, ypred, average='micro'))
    auc.append(roc_auc_score(y_test, clf.predict_proba(X_test),multi_class='ovr'))

In [8]:
np.mean(f1)

0.6336867088607596

In [9]:
np.mean(auc)

0.7867323451307359

In [ ]:
X_1.shape

In [22]:
from sklearn.preprocessing import label_binarize
y_1= label_binarize(y_1, classes=[0, 1, 2])

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_1, y_1, test_size=0.33, random_state=42)
clf = RandomForestClassifier(n_estimators=400).fit(X_train, y_train)
y_score = clf.predict_proba(X_test)

AttributeError: 'list' object has no attribute 'shape'

In [14]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

n_classes = 3
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

TypeError: list indices must be integers or slices, not tuple

### Model Building

In [ ]:
columns = list(range(0,11))+list(range(12,24))+list(range(25,36))

In [ ]:
df = df.iloc[:,columns].dropna()

In [ ]:
df.info()

In [ ]:
df.to_csv("ssss.csv")

In [ ]:
feature = list(range(0,22)) + list(range(28,34))

In [ ]:
y = df.iloc[:,[24,25,26,27]]

In [ ]:
X = df.iloc[:,feature]

In [ ]:
y

In [ ]:
X.info()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scale = scaler.transform(X)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_scale, list(y.iloc[:,0]))
reg.score(X_scale, list(y.iloc[:,0]))

In [ ]:
reg = LinearRegression().fit(X_scale, list(y.iloc[:,1]))
reg.score(X_scale, list(y.iloc[:,1]))

In [ ]:
reg = LinearRegression().fit(X_scale, list(y.iloc[:,2]))
reg.score(X_scale, list(y.iloc[:,2]))

In [ ]:
reg = LinearRegression().fit(X_scale, list(y.iloc[:,3]))
reg.score(X_scale, list(y.iloc[:,3]))

In [ ]:
X.reset_index(drop=True,inplace=True)

In [ ]:
vec = pd.read_csv('vec_pred.csv')
frame = [X,vec]
X_new = pd.concat(frame,axis=1)

In [ ]:
y.reset_index(drop=True,inplace=True)

In [ ]:
frame1 = [X_new,y]
df_output = pd.concat(frame1,axis=1)

In [ ]:
df_output.to_csv("output.csv")

In [ ]:
fea = list(range(0,28))+[28]
scaler = preprocessing.StandardScaler().fit(X_new.iloc[:,fea])
X_new_scale = scaler.transform(X_new.iloc[:,fea])
reg = LinearRegression().fit(X_new_scale, list(y.iloc[:,0]))
reg.score(X_new_scale, list(y.iloc[:,0]))

In [ ]:
fea = list(range(0,28))+[29]
scaler = preprocessing.StandardScaler().fit(X_new.iloc[:,fea])
X_new_scale = scaler.transform(X_new.iloc[:,fea])
reg = LinearRegression().fit(X_new_scale, list(y.iloc[:,1]))
reg.score(X_new_scale, list(y.iloc[:,1]))

In [ ]:
fea = list(range(0,28))+[30]
scaler = preprocessing.StandardScaler().fit(X_new.iloc[:,fea])
X_new_scale = scaler.transform(X_new.iloc[:,fea])
reg = LinearRegression().fit(X_new_scale, list(y.iloc[:,2]))
reg.score(X_new_scale, list(y.iloc[:,2]))

In [ ]:
fea = list(range(0,28))+[31]
scaler = preprocessing.StandardScaler().fit(X_new.iloc[:,fea])
X_new_scale = scaler.transform(X_new.iloc[:,fea])
reg = LinearRegression().fit(X_new_scale, list(y.iloc[:,3]))
reg.score(X_new_scale, list(y.iloc[:,3]))

Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
scores = cross_validate(reg, X_new_scale, list(y.iloc[:,3]), cv=5,scoring = ['r2','neg_mean_squared_error','explained_variance'])

In [ ]:
y_1= np.array(y.iloc[:,3])

In [ ]:
y_1[80]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
kf = KFold(n_splits=10)
X_1 = X_new_scale
y_1= np.array(y.iloc[:,3])
mse_lst=[]
for train_index, test_index in kf.split(X_1):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_1[train_index], X_1[test_index]
    y_train, y_test = y_1[train_index], y_1[test_index]
    reg = LinearRegression().fit(X_train, y_train)
    ypred = reg.predict(X_test)
    mse_lst.append(mean_squared_error(y_test,ypred))

In [ ]:
mse_lst

In [ ]:
np.mean(mse_lst)